In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import zipfile
from subprocess import check_output

for dirname, _, filenames in os.walk('/kaggle/input/instacart-market-basket-analysis/'):
    for filename in filenames:        
        archive = zipfile.ZipFile(os.path.join(dirname, filename), mode='r')
        archive.extractall(path="/kaggle/working")
        archive.close()

print(check_output(["ls", "../working"]).decode("utf8"))

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        col_type2 = df[col].dtype.name
        
        if ((col_type != object) and (col_type2 != 'category')):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
orders_df = reduce_mem_usage(pd.read_csv('../working/orders.csv'))
order_products__prior_df = reduce_mem_usage(pd.read_csv('../working/order_products__prior.csv'))
order_products__train_df = reduce_mem_usage(pd.read_csv('../working/order_products__train.csv'))
products_df = reduce_mem_usage(pd.read_csv('../working/products.csv'))
department_df = reduce_mem_usage(pd.read_csv('../working/departments.csv'))
aisles_df = reduce_mem_usage(pd.read_csv('../working/aisles.csv'))

In [ ]:
variables = ['variables','orders_df','order_products__prior_df','order_products__train_df','products_df','department_df','aisles_df']

Check the columns in different dataframes

In [ ]:
len(orders_df)

In [ ]:
orders_df.eval_set.dtypes

In [ ]:
sum(orders_df.order_id.value_counts().values)

In [ ]:
orders_df.eval_set.value_counts()

In [ ]:
orders_df.user_id.nunique()

In [ ]:
orders_df.groupby('eval_set')['user_id'].nunique()

In [ ]:
temp = orders_df.groupby(['user_id','eval_set'])['order_id'].nunique().reset_index()

In [ ]:
min(temp.groupby(['user_id'])['order_id'].sum())

In [ ]:
max(temp.groupby(['user_id'])['order_id'].sum())

In [ ]:
prior_df = orders_df[orders_df.eval_set == 'prior']

In [ ]:
temp = reduce_mem_usage(pd.merge(left=prior_df, right=order_products__prior_df, on='order_id'))
temp.head()

In [ ]:
temp2 = reduce_mem_usage(pd.merge(left=products_df, right=aisles_df, on='aisle_id'))
temp3 = reduce_mem_usage(pd.merge(left=temp2, right=department_df, on='department_id'))
temp3

In [ ]:
final_prior_df = reduce_mem_usage(pd.merge(left=temp, right=temp3, on='product_id'))
final_prior_df.head()

In [ ]:
ord_count_per_prod = reduce_mem_usage(final_prior_df[['order_id','user_id','product_name']].groupby('product_name').nunique().reset_index())
ord_count_per_prod.head()

In [ ]:
#sns.barplot(data=ord_count_per_prod, x='product_name', y='user_id')
#plt.show()

In [ ]:
final_prior_df.isnull().sum()/len(final_prior_df)

In [ ]:
final_prior_df[final_prior_df.days_since_prior_order.isnull()]['order_number'].nunique()
#gc.collect()

In [ ]:
variables.append('final_prior_df')

In [ ]:
def my_reset(varnames):
    """
    varnames are what you want to keep
    """
    globals_ = globals()
    to_save = {v: globals_[v] for v in varnames}
    to_save['my_reset'] = my_reset  # lets keep this function by default
    del globals_
    get_ipython().magic("reset")
    globals().update(to_save)

In [ ]:
my_reset(variables)

In [ ]:
final_prior_df.head()

In [ ]:
final_prior_df.groupby(['product_id','user_id'])['order_id'].count()

In [ ]:
aisles_df.aisle.value_counts()

In [ ]:
my_reset(variables)

In [ ]:
final_prior_df.columns

In [ ]:
final_prior_df.order_hour_of_day.nunique()

In [ ]:
final_prior_df.order_dow.nunique()

In [ ]:
dummies_df = pd.get_dummies(data=final_prior_df, prefix=['Day','Hour'], columns=['order_dow','order_hour_of_day'], drop_first=True)
dummies_df.head(10)

In [ ]:
dummies_df.columns

In [ ]:
user_prod_df = dummies_df.groupby(['user_id','product_id']).agg({'order_id':'nunique',
                                                                 'days_since_prior_order':'mean',
                                                                 'reordered':'max',
                                                                 'Day_1':'sum',
                                                                 'Day_2':'sum',
                                                                 'Day_3':'sum',
                                                                 'Day_4':'sum',
                                                                 'Day_5':'sum',
                                                                 'Day_6':'sum',
                                                                 'Hour_1':'sum',
                                                                 'Hour_2':'sum',
                                                                 'Hour_3':'sum',
                                                                 'Hour_4':'sum',
                                                                 'Hour_5':'sum',
                                                                 'Hour_6':'sum',
                                                                 'Hour_7':'sum',
                                                                 'Hour_8':'sum',
                                                                 'Hour_9':'sum',
                                                                 'Hour_10':'sum',
                                                                 'Hour_11':'sum',
                                                                 'Hour_12':'sum',
                                                                 'Hour_13':'sum',
                                                                 'Hour_14':'sum',
                                                                 'Hour_15':'sum',
                                                                 'Hour_16':'sum',
                                                                 'Hour_17':'sum',
                                                                 'Hour_18':'sum',
                                                                 'Hour_19':'sum',
                                                                 'Hour_20':'sum',
                                                                 'Hour_21':'sum',
                                                                 'Hour_22':'sum',
                                                                 'Hour_23':'sum'
                                                                }).reset_index()
user_prod_df.head(10)

In [ ]:
user_purchase_df = dummies_df.groupby(['user_id']).agg({         'order_id':'nunique',
                                                                 'product_id': 'nunique',
                                                                 'days_since_prior_order':'mean',
                                                                 'reordered':'sum',
                                                                 'Day_1':'sum',
                                                                 'Day_2':'sum',
                                                                 'Day_3':'sum',
                                                                 'Day_4':'sum',
                                                                 'Day_5':'sum',
                                                                 'Day_6':'sum',
                                                                 'Hour_1':'sum',
                                                                 'Hour_2':'sum',
                                                                 'Hour_3':'sum',
                                                                 'Hour_4':'sum',
                                                                 'Hour_5':'sum',
                                                                 'Hour_6':'sum',
                                                                 'Hour_7':'sum',
                                                                 'Hour_8':'sum',
                                                                 'Hour_9':'sum',
                                                                 'Hour_10':'sum',
                                                                 'Hour_11':'sum',
                                                                 'Hour_12':'sum',
                                                                 'Hour_13':'sum',
                                                                 'Hour_14':'sum',
                                                                 'Hour_15':'sum',
                                                                 'Hour_16':'sum',
                                                                 'Hour_17':'sum',
                                                                 'Hour_18':'sum',
                                                                 'Hour_19':'sum',
                                                                 'Hour_20':'sum',
                                                                 'Hour_21':'sum',
                                                                 'Hour_22':'sum',
                                                                 'Hour_23':'sum'
                                                                }).reset_index()
user_purchase_df.head(10)

In [ ]:
product_purchase_df = dummies_df.groupby(['product_id']).agg({   'order_id':'nunique',
                                                                 'user_id': 'nunique',
                                                                 'days_since_prior_order':'mean',
                                                                 'reordered':'sum',
                                                                 'Day_1':'sum',
                                                                 'Day_2':'sum',
                                                                 'Day_3':'sum',
                                                                 'Day_4':'sum',
                                                                 'Day_5':'sum',
                                                                 'Day_6':'sum',
                                                                 'Hour_1':'sum',
                                                                 'Hour_2':'sum',
                                                                 'Hour_3':'sum',
                                                                 'Hour_4':'sum',
                                                                 'Hour_5':'sum',
                                                                 'Hour_6':'sum',
                                                                 'Hour_7':'sum',
                                                                 'Hour_8':'sum',
                                                                 'Hour_9':'sum',
                                                                 'Hour_10':'sum',
                                                                 'Hour_11':'sum',
                                                                 'Hour_12':'sum',
                                                                 'Hour_13':'sum',
                                                                 'Hour_14':'sum',
                                                                 'Hour_15':'sum',
                                                                 'Hour_16':'sum',
                                                                 'Hour_17':'sum',
                                                                 'Hour_18':'sum',
                                                                 'Hour_19':'sum',
                                                                 'Hour_20':'sum',
                                                                 'Hour_21':'sum',
                                                                 'Hour_22':'sum',
                                                                 'Hour_23':'sum'
                                                                }).reset_index()
product_purchase_df.head(10)

In [ ]:
temp = pd.merge(left=user_prod_df,  right=user_purchase_df, on='user_id', suffixes=('','_user'))
temp.head(10)

In [ ]:
features_df = pd.merge(left=temp,  right=product_purchase_df, on='product_id', suffixes=('','_prod'))
features_df.head(10)

In [ ]:
features_df.shape

In [ ]:
features_df.info()

In [ ]:
variables = ['features_df']
my_reset(variables)

In [ ]:
reduce_features_df = reduce_mem_usage(features_df)

In [ ]:
variables = ['reduce_features_df']
my_reset(variables)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(reduce_features_df, test_size=0.3, random_state=100)
print(train_df.shape)
print(test_df.shape)

In [ ]:
variables = ['train_df','test_df']
my_reset(variables)

In [ ]:
train_sparse_df = train_df.to
variables = ['train_sparse_df']
my_reset(variables)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features_scaled_df = scaler.fit_transform(train_df)

In [ ]:
variables = ['features_scaled_df']
my_reset(variables)

In [ ]:
#import pandas as pd
#features_df = pd.read_csv('../working/features.csv')

In [ ]:
#features_df.columns

In [ ]:
#features_df[['Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6']].corr()